In [ ]:
import pathlib
import json

import pandas as pd

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
import pymedphys._utilities.filehash

from prototyping import *

In [ ]:
logfile_directory = pathlib.Path(r'D:\LinacLogFiles')
monaco_directory = pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical')
output_directory = pathlib.Path(r'S:\Physics\Patient Specific Logfile Fluence')

pdf_directory = pathlib.Path(r'P:\Scanned Documents\RT\PhysChecks\Logfile PDFs')

In [ ]:
patient_id = '013029'

percent_deviation = 2
mm_dist_threshold = 0.5

In [ ]:
with pymedphys.mosaiq.connect('msqsql') as cursor:
    fields = get_patient_fields(cursor, patient_id)
    
valid_fields = fields[(fields['field_version'] == 0) & (fields['monitor_units'] != 0)]
valid_fields

In [ ]:
all_tel_paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/tel.1'))
all_tel_paths

In [ ]:
monaco_plan_name = 'WHOLEBRAIN'
mosaiq_site = 'WHOLE BRAIN'

In [ ]:
paths = list(logfile_directory.joinpath('indexed').glob(f'*/{patient_id}_*/*/*/*/*.trf'))
file_hashes = [pymedphys._utilities.filehash.hash_file(path) for path in paths]

In [ ]:
with open(logfile_directory.joinpath('index.json')) as f:
    index = json.load(f)


index_records = [
    index[key] for key in file_hashes
]

In [ ]:
# index_records

In [ ]:
field_ids = [record['delivery_details']['field_id'] for record in index_records]
# field_ids

In [ ]:
fields_at_site = valid_fields[valid_fields['site'] == mosaiq_site]
fields_at_site

In [ ]:
logfiles_to_check = [path for path, field_id in zip(paths, field_ids) if field_id in fields_at_site['field_id'].array]
logfiles_to_check

In [ ]:
tel_paths = list(monaco_directory.glob(f'*~{patient_id}/plan/{monaco_plan_name}/tel.1'))
tel_paths

In [ ]:
for tel_file in tel_paths:
    markdown_print(f"## `{tel_file}`")

    try:
        mudensity_tel = get_mu_density_from_file(tel_file)

        for trf_file in logfiles_to_check:
            markdown_print(f"### `{trf_file}`")

            mudensity_trf = get_mu_density_from_file(trf_file)

            gamma = calc_gamma(to_tuple(mudensity_tel), to_tuple(mudensity_trf))

            results_dir = output_directory.joinpath(patient_id, tel_file.parent.name, trf_file.stem)
            results_dir.mkdir(exist_ok=True, parents=True)

            header_text = (
                f"Patient ID: {patient_id}\n"
                f"Plan Name: {tel_file.parent.name}\n"
            )

            footer_text = (
                f"tel.1 file path: {str(tel_file)}\n"
                f"trf file path: {str(trf_file)}\n"
                f"results path: {str(results_dir)}"
            )

            png_filepath = str(results_dir.joinpath("result.png").resolve())
            pdf_filepath = str(pdf_directory.joinpath(f"{patient_id}.pdf").resolve())

            fig = plot_and_save_results(
                mudensity_tel, mudensity_trf, 
                gamma, png_filepath, pdf_filepath, 
                header_text=header_text, footer_text=footer_text
            )
            
            fig.tight_layout()
            plt.savefig(png_filepath, dpi=300)
            plt.show()

            !magick convert "{png_filepath}" "{pdf_filepath}"


    except Exception as e:
        traceback.print_exc()